In [2]:
import os
os.chdir("/scratch/hm62/hl4138/polyBERT/polyBERT/")
from evaluate_ds import *

In [3]:
# Load test dataset
file_path = 'data/generated_polymer_smiles_dev.txt'
csv_file = "masking_evaluation.csv"

with open(file_path, 'r') as file:
    psmiles_strings = [line.strip() for line in file]

psmiles_strings = psmiles_strings[:20]

In [22]:
class DebertaMLM(L.LightningModule):
    def __init__(self, model_path, tokeniser):
        super().__init__()
        self.model = DebertaV2ForMaskedLM.from_pretrained(model_path)

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask)

        
    

In [23]:
size = '1M'
tokeniser = DebertaV2Tokenizer(f"spm_{size}.model",f"spm_{size}.vocab")
model = DebertaMLM(f'model_{size}_final/',tokeniser)

In [24]:
polyBERT_1M = polyBERT(model,tokeniser)
# Mask 15% of tokens of each string in test data
masked_psmiles, ground_truth = create_masked_test_set(tokeniser,psmiles_strings)

# Tokenize the sentences
inputs = tokeniser(masked_psmiles, return_tensors='pt', padding=True)

# Create a DataLoader to batch inputs
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
dataloader = DataLoader(dataset, batch_size=64)

all_predicted_token_ids = []
all_true_token_ids = tokeniser.convert_tokens_to_ids(ground_truth)

In [25]:
trainer = Trainer()
predictions = trainer.predict(model, dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: forward() missing 1 required positional argument: 'attention_mask'